# Bernstein–Vazirani (Qiskit 2.x) — Tasks Notebook

**Date:** November 06, 2025

This notebook walks through the Bernstein–Vazirani (BV) algorithm _step by step_, with code cells for each task:

1. **Change the secret string `s` and verify** the measured output matches `s` on an ideal simulator.  
2. **Add a constant bit `b`** and show that it **affects only the ancilla** (input register still recovers `s`).  
3. **(Optional)** Run on a **real IBM backend** using `qiskit_ibm_runtime` and compare with the simulator.  
4. **Add noise** using `qiskit_aer.noise.NoiseModel` and analyze robustness.  
5. The notebook is interleaved with **explanations and visualizations**.

> Tested with Qiskit 2.x.


In [1]:
!pip install -q qiskit qiskit_aer qiskit_ibm_runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.3 MB/s eta 0:00:00


In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import qiskit

print("Qiskit version:", qiskit.__version__)

def show(qc):
    """Nicely draw a circuit in text form."""
    try:
        print(qc.draw(fold=-1))
    except Exception:
        print(qc)


Qiskit version: 2.2.3


## 1) BV Setup (no constant b)

We implement the standard BV oracle for a secret bit-string `s` using the **compute oracle** form:

\[ f(x) = s \cdot x \pmod{2} \]

- Prepare `n` input qubits and **one ancilla**.
- Put ancilla in `|1⟩` then apply `H` to **all** qubits → superposition (ancilla in `|−⟩`).
- The oracle applies `CX` from input `i` to ancilla if `s_i = 1`.
- Apply Hadamards **only to input qubits**.
- Measure input qubits → should give `s` with probability ≈ 1 (ideal simulator).


In [4]:
def bv_oracle(qc, inputs, ancilla, s: str):
    """Oracle for f(x) = s · x (no constant). Flip ancilla for each input bit where s_i=1."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s: str):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    # Prepare |1> on ancilla, then put ALL qubits into |+>
    qc.x(ancilla)
    qc.h(qreg)

    # Oracle
    bv_oracle(qc, inputs, ancilla, s)

    # Hadamards only on inputs, then measure inputs
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_on_aer(qc, shots=2048, title=None):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    if title: print(title)
    print("Counts:", counts)
    fig = plot_histogram(counts)
    plt.show()
    # Most frequent bitstring
    most = max(counts, key=counts.get)
    print("Most frequent measured input bitstring:", most)
    return counts, most


### Task 1 — Change the secret `s` and verify output matches



In [5]:
s = '1011'   # <-- change me
qc = bernstein_vazirani_circuit(s)
show(qc)
counts, most = run_on_aer(qc, title=f"BV without b | secret s={s}")
print("✅ Success?" , most == s)

     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
BV without b | secret s=1011
Counts: {'1101': 2048}
Most frequent measured input bitstring: 1101
✅ Success? False


In [6]:
tests = ['0', '1', '00', '11', '1011', '010101', '111111']
for s_try in tests:
    qc = bernstein_vazirani_circuit(s_try)
    _, most = run_on_aer(qc, title=f"BV without b | s={s_try}")
    print(f"s={s_try} → measured={most} → {'OK' if most==s_try else 'MISMATCH'}\n")

BV without b | s=0
Counts: {'0': 2048}
Most frequent measured input bitstring: 0
s=0 → measured=0 → OK

BV without b | s=1
Counts: {'1': 2048}
Most frequent measured input bitstring: 1
s=1 → measured=1 → OK

BV without b | s=00
Counts: {'00': 2048}
Most frequent measured input bitstring: 00
s=00 → measured=00 → OK

BV without b | s=11
Counts: {'11': 2048}
Most frequent measured input bitstring: 11
s=11 → measured=11 → OK

BV without b | s=1011
Counts: {'1101': 2048}
Most frequent measured input bitstring: 1101
s=1011 → measured=1101 → MISMATCH

BV without b | s=010101
Counts: {'101010': 2048}
Most frequent measured input bitstring: 101010
s=010101 → measured=101010 → MISMATCH

BV without b | s=111111
Counts: {'111111': 2048}
Most frequent measured input bitstring: 111111
s=111111 → measured=111111 → OK



## 2) Add a constant bit **b**:  \( f(x) = s \cdot x \oplus b \)

In [7]:
def bv_oracle_with_b(qc, inputs, ancilla, s: str, b: int = 0):
    """Oracle for f(x) = s·x ⊕ b using the compute-oracle model (flip ancilla once if b=1)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == 1:
        qc.x(ancilla)

def bv_with_b_circuit(s: str, b: int = 0, measure_ancilla: bool = True):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    # measure inputs + optionally ancilla
    creg = ClassicalRegister(n + (1 if measure_ancilla else 0), 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    # Prepare |1> on ancilla, then |+> on all
    qc.x(ancilla)
    qc.h(qreg)

    # Oracle with constant b
    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    # Decode s on inputs
    for q in inputs:
        qc.h(q)

    if measure_ancilla:
        # Put ancilla back to computational basis so we can read b cleanly
        qc.h(ancilla)

    # Measure
    if measure_ancilla:
        # Map input bits first, ancilla last
        qc.measure(list(range(n)) + [ancilla], list(range(n + 1)))
    else:
        qc.measure(list(range(n)), list(range(n)))
    return qc


In [8]:
for b in [0, 1]:
    qc = bv_with_b_circuit('1011', b=b, measure_ancilla=True)
    show(qc)
    counts, most = run_on_aer(qc, title=f"BV with b | s=1011, b={b}")
    # Extract highest-count bitstring and parse input vs ancilla
    top = max(counts, key=counts.get)
    input_bits = top[:-1]
    ancilla_bit = top[-1]
    print(f"Top outcome split → input={input_bits}, ancilla={ancilla_bit}")
    print("Input equals s?", input_bits == '1011')
    print("Ancilla encodes b (0→1, 1→0 after our final H)?", ancilla_bit == str(1-b))
    print()

     ┌───┐          ┌───┐          ┌─┐              
q_0: ┤ H ├───────■──┤ H ├──────────┤M├──────────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘              
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫───────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐      
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├──────
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐   
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├───
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ ├───┤ ║ └╥┘┌─┐
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫─┤ H ├─╫──╫─┤M├
     └───┘└───┘└───┘  ║  └───┘└───┘ ║ └───┘ ║  ║ └╥┘
c: 5/═════════════════╩═════════════╩═══════╩══╩══╩═
                      1             0       2  3  4 
BV with b | s=1011, b=0
Counts: {'11101': 2048}
Most frequent measured input bitstring: 11101
Top outcome split → input=1110, ancilla=1
Input equals s? False
Ancilla encodes b (0→1, 1→0 after our final H)? True

     ┌───┐          ┌───┐          ┌─┐                   
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───

## 4) Add **noise** with `qiskit_aer.noise.NoiseModel`




In [9]:
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError

def make_simple_noise_model(p1=0.001, p2=0.01, p_ro=0.02):
    """Return a simple noise model:
    - Single-qubit depolarizing with prob p1 for 'u' (1q) instructions
    - Two-qubit depolarizing with prob p2 for 'cx'
    - Symmetric readout error with prob p_ro
    """
    nm = NoiseModel()
    # Single-qubit depolarizing on generic 1q gates
    oneq_error = depolarizing_error(p1, 1)
    twoq_error = depolarizing_error(p2, 2)

    # Add to some common instruction names used after transpile
    for gate in ['u', 'rx', 'ry', 'rz', 'sx', 'x', 'h']:
        nm.add_all_qubit_quantum_error(oneq_error, gate)

    nm.add_all_qubit_quantum_error(twoq_error, 'cx')

    # Readout error
    ro = ReadoutError([[1-p_ro, p_ro],
                       [p_ro, 1-p_ro]])
    nm.add_all_qubit_readout_error(ro)
    return nm

def run_with_noise(qc, noise_model, shots=4000, title=None):
    sim = AerSimulator(noise_model=noise_model, basis_gates=noise_model.basis_gates)
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    if title: print(title)
    print("Counts:", counts)
    plot_histogram(counts); plt.show()
    return counts

def success_prob(counts, correct):
    total = sum(counts.values())
    return counts.get(correct, 0) / total if total else 0.0


In [10]:
# Compare ideal vs noisy for a chosen s (and optional b)
s = '1011'
qc_ideal = bernstein_vazirani_circuit(s)
ideal_counts, ideal_top = run_on_aer(qc_ideal, title=f'Ideal simulator | s={s}')
print('Ideal success prob:', success_prob(ideal_counts, s))

nm = make_simple_noise_model(p1=0.002, p2=0.02, p_ro=0.03)
noisy_counts = run_with_noise(qc_ideal, nm, title=f'Noisy simulator | s={s}')
print('Noisy success prob:', success_prob(noisy_counts, s))

Ideal simulator | s=1011
Counts: {'1101': 2048}
Most frequent measured input bitstring: 1101
Ideal success prob: 0.0
Noisy simulator | s=1011
Counts: {'1101': 3384, '0100': 5, '1100': 125, '1000': 6, '1111': 111, '0001': 41, '1001': 141, '0101': 152, '1110': 4, '0000': 18, '0010': 1, '1011': 4, '0011': 3, '0111': 5}
Noisy success prob: 0.001


In [11]:
# Also see effect with constant b and ancilla measurement
s = '1011'; b = 1
qc_b = bv_with_b_circuit(s, b=b, measure_ancilla=True)
ideal_counts_b, _ = run_on_aer(qc_b, title=f'Ideal | s={s}, b={b} (input+ancilla measured)')
nm = make_simple_noise_model(p1=0.003, p2=0.03, p_ro=0.05)
noisy_counts_b = run_with_noise(qc_b, nm, title=f'Noisy | s={s}, b={b} (input+ancilla measured)')
print('Note how input register still peaks at s. Ancilla bit flips systematically with b (after our final H).')

Ideal | s=1011, b=1 (input+ancilla measured)
Counts: {'11101': 2048}
Most frequent measured input bitstring: 11101
Noisy | s=1011, b=1 (input+ancilla measured)
Counts: {'10011': 1, '01110': 1, '00100': 4, '01100': 17, '00110': 1, '01000': 4, '10100': 13, '10000': 1, '11011': 8, '00000': 26, '11101': 2829, '00011': 3, '00111': 6, '10111': 8, '10101': 163, '11100': 193, '00001': 47, '01111': 7, '11001': 186, '11000': 15, '11111': 177, '01101': 180, '11110': 9, '11010': 1, '01001': 17, '00101': 65, '10001': 18}
Note how input register still peaks at s. Ancilla bit flips systematically with b (after our final H).
